In [1]:
import numpy as np
import math
import pandas as pd

In [2]:
c = 3.0e18     # speed of light in Angstrom/s

# Luminosity distances from Ned's Cosmo Calc:  H0= 70, omegaM= 0.27, omegaV= 0.73
# formulas at-- http://www.astro.ucla.edu/~wright/Distances_details.gif 

dic = {'1025+390':1940.5, '1037+30':417.2, '1128+455':2217.0, '1201+394':2488.2, 
                                                '1203+645':2004.1, '1221-423':826.8}  # (in Mpc)

#(1 Mpc =  3.08e24 cm)

# array to store lum. dist. values in cm
d = np.zeros(6, dtype=np.float32)

i = 0
for source in dic:
    d[i] = dic[source] * 3.08e24
    i = i+1


In [27]:
def hst_sfr(df):

    hst_mAB = df['hst_uv'].values
    hst_Fnu = [ 10**(- (k + 48.6) / 2.5) for k in hst_mAB ]
    hst_Lnu = [(4 * math.pi * d[i]**2) * hst_Fnu[i] for i in range (len(d))]
    
    hst_SFR = [round(1.4e-28 * hst_Lnu[i],2) for i in range (len(d))]
    
    return hst_SFR

In [4]:
def galex_sfr(df):

    galex_mAB = df['nuv'].values
    galex_Fnu = [ 10**(- (k + 48.6) / 2.5) for k in galex_mAB ]
    galex_Lnu = [(4 * math.pi * d[i]**2) * galex_Fnu[i] for i in range (len(d))]
    galex_SFR = [round(1.08e-28 * galex_Lnu[i],2) for i in range (len(d))]
    
    return galex_SFR

In [5]:
def infrared_sfr(df):

    mVega = data['w4'].values
    mAB = mVega + 6.62
    w4_Fnu = [ 10**(- (k + 48.6) / 2.5) for k in mAB ]
    w4_nu_eff = 1.3456e13  # nu for effective wavelength W4 band
    w4_L = [(4 * math.pi * d[i]**2) * w4_nu_eff * w4_Fnu[i] for i in range (len(d))]    
                                     # nuFnu -> dimensions of flux (ergs/sec)
    L_sun = 3.85e33    #ergs/sec
    w4_Ls = [w4_L[i] / L_sun for i in range (len(w4_L))]    # L_w4 in ergs/sec
    
    
    # Star formation relation : Rieke et al. 2009
    #        -- given relation for Spitzer/24micron band, but can be used for WISE 22 micron too
    
    w4_SFR, w4_SFR_2 = np.zeros(6, dtype=np.float32), np.zeros(6, dtype=np.float32)
    
    for i in range (len(w4_Ls)):
        if w4_Ls[i] <= 1.3e10:
            w4_SFR[i] = round(7.8e-10 * w4_Ls[i], 2)
        else:
            w4_SFR[i] = round(7.8e-10 * w4_Ls[i] * (7.76e-11 * w4_Ls[i])**0.048, 2)
        
    # Wu et al. 2005 relation
    for i in range (len(w4_Ls)):
        w4_SFR_2[i] = round(6.66e-8 * w4_Ls[i], 2)
        
    
    return w4_L, w4_SFR, w4_SFR_2


In [6]:
def mixed_sfr(df, L_w4):

    mHa = data['halpha'].values
    Halpha_Fnu = [ 10**(- (k + 48.6) / 2.5) for k in mHa ]
    Ha_nu_eff = c / 6563  # nu for effective wavelength in Angstrom
    Ha_L = [(4 * math.pi * d[i]**2) * Ha_nu_eff * Halpha_Fnu[i] for i in range (len(d))] 
    
    Ha_SFR = np.zeros(len(Ha_L), dtype=np.float32)
    # Ha star formation relation: Kennicutt et al. 1998 
    Ha_SFR = [round(7.9e-42 * Ha_L[i], 2) for i in range (len(Ha_L))]
    
    
    moII = data['oII'].values
    oII_Fnu = [ 10**(- (k + 48.6) / 2.5) for k in moII ]
    oII_nu_eff = c / 4970 # nu for average effective wavelength in Angstrom
    oII_L = [(4 * math.pi * d[i]**2) * oII_nu_eff * oII_Fnu[i] for i in range (len(d))] 

    oII_SFR = np.zeros(len(Ha_L), dtype=np.float32)
    # [OII] star formation relation: Kennicutt et al. 1998 
    oII_SFR = [round(1.4e-41 * oII_L[i], 2) for i in range (len(oII_L))]
    
    
    # Kennicutt et al. 2009
    SFR_w4_Ha = [round(7.9e-42 * (Ha_L[i] + 0.020*L_w4[i]), 2) for i in range (len(d))]
    SFR_w4_OII = [round(8.1e-42 * (oII_L[i] + 0.029*L_w4[i]), 2) for i in range (len(d))]
    
    
    return Ha_SFR, oII_SFR, SFR_w4_Ha, SFR_w4_OII
    

In [28]:
data = pd.read_csv('gal_corr_sfr_mags.csv', header=[0])
print(data)

hst_sfr = hst_sfr(data)
nuv_sfr = galex_sfr(data)
w4band_L, w4_sfr, w4_sfr_2 = infrared_sfr(data)
ha_sfr, oII_sfr, sfr_w4_ha, sfr_w4_oII = mixed_sfr(data, w4band_L)


    Targets    nuv  hst_uv  halpha    oII    w4
0  1025+390  21.28   22.66   25.09  25.43  8.22
1   1037+30  19.36   19.85   23.79  24.22  7.34
2  1128+455  23.41   22.91   24.52  26.60  8.01
3  1201+394  22.46   24.57     NaN  27.66  8.78
4  1203+645  23.15   22.74   23.04  26.21  7.62
5  1221-423  19.66   19.45   24.98  27.82  6.76


In [29]:
import scipy
from tabulate import tabulate

table = [hst_sfr, nuv_sfr, w4_sfr, ha_sfr, oII_sfr, sfr_w4_ha, sfr_w4_oII]
print(tabulate(table))

#rowwise stack

-----  ----  -----  ------  ------  -----
 1.97  1.21   2.04    0.56    1.95   6.87
 5.41  1.47   0.99    3       1.03   4.37
55.69  5.35  90.16   54.61  106.32  38.12
 5.42  0.83  11.95  nan      38.18   1.09
 9.27  1.31   4.12    1.95    4.82   0.19
45.57  5     75.55  nan     112.6   29.06
65.06  6.96  96.93   59.73  113.44  41.69
-----  ----  -----  ------  ------  -----


In [9]:
Targets=[]
for key in dic:
    Targets.append(key)

In [30]:
sfr_data = {'Source': Targets, 'hst_uv': hst_sfr, 'nuv': nuv_sfr, 'w4': w4_sfr, 'halpha': ha_sfr, 'oII': oII_sfr, 
            'w4+ha': sfr_w4_ha, 'w4+oII': sfr_w4_oII}
            
#df_new = pd.DataFrame(sfr_data, index=Targets)


#sfr_data = {'Source': Targets, 'SFR$_{HST/UV}$': hst_sfr, 'SFR$_{GALEX/NUV}$': nuv_sfr, 'SFR$_{W4}$ ': w4_sfr, r'SFR$_{H\alpha}$': ha_sfr, 
#            'SFR$_{[OII]}$': oII_sfr, r'SFR$_{W4+H\alpha}$': sfr_w4_ha, 'SFR$_{W4+[OII]}$': sfr_w4_oII}

names = ['Source','nuv','w4','halpha','oII','w4+ha','w4+oII']
df_new = pd.DataFrame(sfr_data)#, columns=names)

print(df_new)

     Source  hst_uv   nuv          w4  halpha   oII   w4+ha  w4+oII
0  1025+390    1.97  5.41   55.689999    5.42  9.27   45.57   65.06
1   1037+30    1.21  1.47    5.350000    0.83  1.31    5.00    6.96
2  1128+455    2.04  0.99   90.160004   11.95  4.12   75.55   96.93
3  1201+394    0.56  3.00   54.610001     NaN  1.95     NaN   59.73
4  1203+645    1.95  1.03  106.320000   38.18  4.82  112.60  113.44
5  1221-423    6.87  4.37   38.119999    1.09  0.19   29.06   41.69


In [31]:
df_new.replace(np.nan, "\nodata")
#df['column name'] = df['column name'].replace(['old value'], 'new value')

,Source,hst_uv,nuv,w4,halpha,oII,w4+ha,w4+oII
0,1025+390,1.97,5.41,55.689999,5.42,9.27,45.57,65.06
1,1037+30,1.21,1.47,5.350000,0.83,1.31,5.0,6.96
2,1128+455,2.04,0.99,90.160004,11.95,4.12,75.55,96.93
3,1201+394,0.56,3.00,54.610001,\nodata,1.95,\nodata,59.73
4,1203+645,1.95,1.03,106.320000,38.18,4.82,112.6,113.44
5,1221-423,6.87,4.37,38.119999,1.09,0.19,29.06,41.69


In [32]:
print(tabulate(df_new.replace(np.nan, "\\nodata"), tablefmt="latex", headers="keys", 
               showindex=False, numalign="center", stralign="center"))

\begin{tabular}{cccccccc}
\hline
  Source  &  hst\_uv  &  nuv  &   w4   &  halpha  &  oII  &  w4+ha  &  w4+oII  \\
\hline
 1025+390 &   1.97   & 5.41  & 55.69  &   5.42   & 9.27  &  45.57  &  65.06   \\
 1037+30  &   1.21   & 1.47  &  5.35  &   0.83   & 1.31  &   5.0   &   6.96   \\
 1128+455 &   2.04   & 0.99  & 90.16  &  11.95   & 4.12  &  75.55  &  96.93   \\
 1201+394 &   0.56   &   3   & 54.61  & \textbackslash{}nodata  & 1.95  & \textbackslash{}nodata &  59.73   \\
 1203+645 &   1.95   & 1.03  & 106.32 &  38.18   & 4.82  &  112.6  &  113.44  \\
 1221-423 &   6.87   & 4.37  & 38.12  &   1.09   & 0.19  &  29.06  &  41.69   \\
\hline
\end{tabular}
